<a href="https://colab.research.google.com/github/Pathway2008/Loan_rating/blob/main/except_col.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/MyDrive/loan_rating/train.csv')
test = pd.read_csv('/content/drive/MyDrive/loan_rating/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/loan_rating/sample_submission.csv')

In [ ]:
train.drop(['ID'],axis=1,inplace=True)
test.drop(['ID'],axis=1,inplace=True)

In [ ]:
train = train[train['주택소유상태'] != 'ANY']
train['근로기간'] = train['근로기간'].replace('< 1 year', '<1 year')
train['근로기간'] = train['근로기간'].replace('1 year', '1 years')
train['근로기간'] = train['근로기간'].replace('3', '3 years')
train['근로기간'] = train['근로기간'].replace('10+years', '10+ years')

In [ ]:
test['근로기간'] = test['근로기간'].replace('< 1 year', '<1 year')
test['근로기간'] = test['근로기간'].replace('1 year', '1 years')
test['근로기간'] = test['근로기간'].replace('3', '3 years')
test['근로기간'] = test['근로기간'].replace('10+years', '10+ years')

In [ ]:
train['원금/대출'] = train['총상환원금']/train['대출금액']
train['이자/대출'] = train['총상환이자']/train['대출금액']
train['소득/대출'] = train['연간소득'] / train['대출금액']
train['연체계좌/총계좌'] = train['연체계좌수'] / train['총계좌수']

train['대출월금액'] = train.apply(lambda row: row['대출금액'] / 36 if row['대출기간'] == 0 else row['대출금액'] / 60, axis=1)
train['이자/소득'] = train.apply(lambda row: row['총상환이자'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)
train['원금/소득'] = train.apply(lambda row: row['총상환원금'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)
train['이자/원금'] = train.apply(lambda row: row['총상환이자'] / row['총상환원금'] if row['총상환원금'] != 0 else 0, axis=1)

In [ ]:
test['원금/대출'] = test['총상환원금'] / test['대출금액']
test['이자/대출'] = test['총상환이자'] / test['대출금액']
test['소득/대출'] = test['연간소득'] / test['대출금액']
test['연체계좌/총계좌'] = test['연체계좌수'] / test['총계좌수']

test['대출월금액'] = test.apply(lambda row: row['대출금액'] / 36 if row['대출기간'] == 0 else row['대출금액'] / 60, axis=1)
test['이자/소득'] = test.apply(lambda row: row['총상환이자'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)
test['원금/소득'] = test.apply(lambda row: row['총상환원금'] / row['연간소득'] if row['연간소득'] != 0 else 0, axis=1)
test['이자/원금'] = test.apply(lambda row: row['총상환이자'] / row['총상환원금'] if row['총상환원금'] != 0 else 0, axis=1)

In [ ]:
train['총상환이자**3'] = train['총상환이자']**3
train['총상환원금**3'] = train['총상환원금']**3

test['총상환이자**3'] = test['총상환이자']**3
test['총상환원금**3'] = test['총상환원금']**3

In [ ]:
from sklearn.preprocessing import LabelEncoder
ordinal_features = ['대출기간', '근로기간', '주택소유상태','대출목적']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [ ]:
train['대출등급'] = le.fit_transform(train['대출등급'])

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.3 MB/s eta 0:00:00


In [ ]:
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [ ]:
from sklearn.model_selection import KFold
kf = StratifiedKFold(n_splits=5, shuffle=False)

In [ ]:
from sklearn.model_selection import train_test_split
X = train.drop(['대출등급','총상환원금','총상환이자','연체계좌수','총계좌수','연체계좌/총계좌'], axis=1)
y = train['대출등급']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3, random_state = 42)

In [ ]:
# XGBoost
xgb_model = xgb.XGBClassifier()#objective = 'multi:softmax', eval_metric = 'logloss'
xgb_model.fit(X_train, y_train)
xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
xgb_predictions = xgb_model.predict(X_test)
xgb_macro = f1_score(y_test, xgb_predictions, average='macro')
# print(f'XGBoost test F1 Macro: {xgb_macro}\n')


# CatBoost
cat_model = CatBoostClassifier(random_state=42, verbose=False)
cat_model.fit(X_train, y_train)
cat_predictions = cat_model.predict(X_test)
cat_scores = cross_val_score(cat_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
cat_macro = f1_score(y_test, cat_predictions, average='macro')
# print(f'Cat test F1 Macro: {cat_macro}\n')

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
rf_macro = f1_score(y_test, rf_predictions, average='macro')
# print(f'RF test F1 Macro: {rf_macro}\n')

# LightGBM
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)
lgb_predictions = lgb_model.predict(X_test)
lgb_scores = cross_val_score(lgb_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')
lgb_macro = f1_score(y_test, lgb_predictions, average='macro')
# print(f'RF test F1 Macro: {lgb_macro}\n')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010417 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3135
[LightGBM] [Info] Number of data points in the train set: 67405, number of used features: 18
[LightGBM] [Info] Start training from score -1.747717
[LightGBM] [Info] Start training from score -1.206424
[LightGBM] [Info] Start training from score -1.248802
[LightGBM] [Info] Start training from score -1.975557
[LightGBM] [Info] Start training from score -2.572111
[LightGBM] [Info] Start training from score -3.897369
[LightGBM] [Info] Start training from score -5.434895
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005159 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3129
[LightGBM

In [ ]:
#일부제외
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}\n\n')

print(f'xgb test F1 Macro: {xgb_macro}')
print(f'cat test F1 Macro: {cat_macro}')
print(f'rf test F1 Macro: {rf_macro}')
print(f'lgb test F1 Macro: {lgb_macro}')

XGBoost Cross-Validation F1 Macro: 0.9157216818245895
CatBoost Cross-Validation F1 Macro: 0.8994425726381035
Random Forest Cross-Validation F1 Macro: 0.9228498453663916
LightGBM Cross-Validation F1 Macro: 0.8852141716017499


xgb test F1 Macro: 0.9123649189118959
cat test F1 Macro: 0.8965524362362728
rf test F1 Macro: 0.9203135360404036
lgb test F1 Macro: 0.8593724195817858


In [ ]:
cat_predictions_flat = np.ravel(cat_predictions)
data = {
    'rf': rf_predictions,
    'xgb': xgb_predictions,
    'cat': cat_predictions_flat,
    'lgb': lgb_predictions,
    'sol': y_test

}
pred_df = pd.DataFrame(data)

In [ ]:
#최빈값 대체
prediction_columns = ['xgb', 'cat', 'rf', 'lgb']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].astype(int)

In [ ]:
stand_blender = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
stand_blender

0.9158172750163569

In [ ]:
#최빈값 대체
prediction_columns = ['xgb', 'cat', 'rf']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].astype(int)

In [ ]:
stand_blender = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
stand_blender

0.9171477574825929

In [ ]:
#최빈값 대체
prediction_columns = ['xgb', 'rf']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].astype(int)

stand_blender = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
stand_blender

0.9153108832710753

In [ ]:
#최빈값 대체
prediction_columns = ['rf','xgb','cat']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].astype(int)

stand_blender = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
stand_blender

0.9171477574825929

In [ ]:
stand_blender = f1_score(pred_df['sol'], pred_df['rf'], average='macro')
stand_blender

0.9203135360404036

In [ ]:
pred_df['rf2'] = pred_df['rf']
pred_df.head()

,rf,xgb,cat,lgb,sol,rf2
72084,2,2,2,2,2,2
42625,0,0,1,1,0,0
1288,1,1,1,1,1,1
19240,0,0,0,0,0,0
10764,2,2,2,2,2,2


In [ ]:
#최빈값 대체
prediction_columns = ['rf','xgb','cat','rf2','lgb']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].astype(int)

stand_blender = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
stand_blender

0.920472756931719

In [ ]:
#일부제외+not hyper
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}\n\n')

print(f'xgb test F1 Macro: {xgb_macro}')
print(f'cat test F1 Macro: {cat_macro}')
print(f'rf test F1 Macro: {rf_macro}')
print(f'lgb test F1 Macro: {lgb_macro}')

XGBoost Cross-Validation F1 Macro: 0.9157216818245895
CatBoost Cross-Validation F1 Macro: 0.8994425726381035
Random Forest Cross-Validation F1 Macro: 0.9228498453663916
LightGBM Cross-Validation F1 Macro: 0.8852141716017499


xgb test F1 Macro: 0.9123649189118959
cat test F1 Macro: 0.8965524362362728
rf test F1 Macro: 0.9203135360404036
lgb test F1 Macro: 0.8593724195817858


In [ ]:
#최빈값 대체
prediction_columns = ['rf','xgb','cat','rf2','lgb']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].astype(int)

stand_blender = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
stand_blender

0.920472756931719

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96293 entries, 0 to 96293
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   대출금액          96293 non-null  int64  
 1   대출기간          96293 non-null  int64  
 2   근로기간          96293 non-null  int64  
 3   주택소유상태        96293 non-null  int64  
 4   연간소득          96293 non-null  int64  
 5   부채_대비_소득_비율   96293 non-null  float64
 6   총계좌수          96293 non-null  int64  
 7   대출목적          96293 non-null  int64  
 8   최근_2년간_연체_횟수  96293 non-null  int64  
 9   총상환원금         96293 non-null  int64  
 10  총상환이자         96293 non-null  float64
 11  총연체금액         96293 non-null  float64
 12  연체계좌수         96293 non-null  float64
 13  대출등급          96293 non-null  int64  
 14  원금/대출         96293 non-null  float64
 15  이자/대출         96293 non-null  float64
 16  소득/대출         96293 non-null  float64
 17  연체계좌/총계좌      96293 non-null  float64
 18  대출월금액         96293 non-nu

In [ ]:
from sklearn.model_selection import train_test_split
X = train.drop(['대출등급','총상환원금','총상환이자','연체계좌수','총계좌수','연체계좌/총계좌','대출목적','주택소유상태'], axis=1)
y = train['대출등급']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3, random_state = 42)

In [ ]:
# XGBoost
xgb_model = xgb.XGBClassifier()#objective = 'multi:softmax', eval_metric = 'logloss'
xgb_model.fit(X_train, y_train)
xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
xgb_predictions = xgb_model.predict(X_test)
xgb_macro = f1_score(y_test, xgb_predictions, average='macro')
# print(f'XGBoost test F1 Macro: {xgb_macro}\n')


# CatBoost
cat_model = CatBoostClassifier(random_state=42, verbose=False)
cat_model.fit(X_train, y_train)
cat_predictions = cat_model.predict(X_test)
cat_scores = cross_val_score(cat_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
cat_macro = f1_score(y_test, cat_predictions, average='macro')
# print(f'Cat test F1 Macro: {cat_macro}\n')

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
rf_macro = f1_score(y_test, rf_predictions, average='macro')
# print(f'RF test F1 Macro: {rf_macro}\n')

# LightGBM
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)
lgb_predictions = lgb_model.predict(X_test)
lgb_scores = cross_val_score(lgb_model, X_train, y_train, cv=kf, scoring='f1_macro')
# print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}')
lgb_macro = f1_score(y_test, lgb_predictions, average='macro')
# print(f'RF test F1 Macro: {lgb_macro}\n')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009794 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3120
[LightGBM] [Info] Number of data points in the train set: 67405, number of used features: 16
[LightGBM] [Info] Start training from score -1.747717
[LightGBM] [Info] Start training from score -1.206424
[LightGBM] [Info] Start training from score -1.248802
[LightGBM] [Info] Start training from score -1.975557
[LightGBM] [Info] Start training from score -2.572111
[LightGBM] [Info] Start training from score -3.897369
[LightGBM] [Info] Start training from score -5.434895
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004794 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3114
[LightGBM

In [ ]:
#except '대출등급','총상환원금','총상환이자','연체계좌수','총계좌수','연체계좌/총계좌','대출목적','주택소유상태'
print(f'XGBoost Cross-Validation F1 Macro: {xgb_scores.mean()}')
print(f'CatBoost Cross-Validation F1 Macro: {cat_scores.mean()}')
print(f'Random Forest Cross-Validation F1 Macro: {rf_scores.mean()}')
print(f'LightGBM Cross-Validation F1 Macro: {lgb_scores.mean()}\n\n')

print(f'xgb test F1 Macro: {xgb_macro}')
print(f'cat test F1 Macro: {cat_macro}')
print(f'rf test F1 Macro: {rf_macro}')
print(f'lgb test F1 Macro: {lgb_macro}')

XGBoost Cross-Validation F1 Macro: 0.9152499520119942
CatBoost Cross-Validation F1 Macro: 0.901147132777371
Random Forest Cross-Validation F1 Macro: 0.9240424254757768
LightGBM Cross-Validation F1 Macro: 0.8771453346090201


xgb test F1 Macro: 0.9141769619691862
cat test F1 Macro: 0.8980736580985142
rf test F1 Macro: 0.923100297885857
lgb test F1 Macro: 0.879022024127868


In [ ]:
cat_predictions_flat = np.ravel(cat_predictions)
data = {
    'rf': rf_predictions,
    'xgb': xgb_predictions,
    'cat': cat_predictions_flat,
    'lgb': lgb_predictions,
    'sol': y_test

}
pred_df = pd.DataFrame(data)

In [ ]:
#최빈값 대체
prediction_columns = ['rf','xgb','cat','lgb']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].astype(int)

stand_blender = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
stand_blender

0.919263900706407

In [ ]:
pred_df['rf2'] = pred_df['rf']

#최빈값 대체
prediction_columns = ['rf','xgb','cat','lgb','rf2']

pred_df['pred'] = pred_df[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_df['pred'] = pred_df['pred'].astype(int)

stand_blender = f1_score(pred_df['sol'], pred_df['pred'], average='macro')
stand_blender

0.9228846868917403

In [ ]:
test = test[['대출금액', '대출기간', '근로기간', '연간소득', '부채_대비_소득_비율', '최근_2년간_연체_횟수', '총연체금액', '원금/대출', '이자/대출', '소득/대출', '대출월금액', '이자/소득', '원금/소득', '이자/원금', '총상환이자**3', '총상환원금**3']]

In [ ]:
xgb_pred = xgb_model.predict(test)
cat_pred = cat_model.predict(test)
rf_pred = rf_model.predict(test)
lgb_pred = lgb_model.predict(test)

In [ ]:
cat_predictions_flat = np.ravel(cat_pred)
data = {
    'rf': rf_pred,
    'xgb': xgb_pred,
    'cat': cat_predictions_flat,
    'lgb': lgb_pred,
    'rf2': rf_pred,
}
pred_blender = pd.DataFrame(data)

In [ ]:
#최빈값 대체
prediction_columns = ['rf','xgb', 'cat', 'rf2', 'lgb']

pred_blender['pred'] = pred_blender[prediction_columns].mode(axis=1, dropna=False).iloc[:, 0]
pred_blender['pred'] = pred_blender['pred'].astype(int)

In [ ]:
sub['대출등급'] = pred_blender['pred']
sub['대출등급'] = sub['대출등급'].map({0: 'A', 1: 'B', 2: 'C',3: 'D',4: 'E',5: 'F',6: 'G'})
sub.head()

In [ ]:
sub.to_csv('2rf_except_col.csv',index=False)

In [ ]:
sub['대출등급'] = pred_blender['rf']
sub['대출등급'] = sub['대출등급'].map({0: 'A', 1: 'B', 2: 'C',3: 'D',4: 'E',5: 'F',6: 'G'})
sub.to_csv('rf_except_col.csv',index=False)